<a href="https://colab.research.google.com/github/GPaz027/Cariety-ML/blob/develop/cariety.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cariety

Este notebook crea un clasificador de imágenes para alimentos utilizando TensorFlow y TensorFlow Hub.

## 1. Problema

* Detectar si la imagen de un alimento se encuentra en especificación o no (es consumible o no).


In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np

# 1. Preparación de los datos

* En esta etapa se convertirán las imágenes a arreglos de NumPy para permitir su preprocesamiento.